<a href="https://colab.research.google.com/github/MasslessAI/narratelab/blob/master/exp/exp_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [1]:
from google.colab import drive

drive.mount('gdrive', force_remount=True)

root_dir = '/content/gdrive/'

gdrive_path = root_dir + 'MyDrive/narratelab/exp_1'

!pip install --upgrade --user numpy
!pip install spacy==3.1.0 sentence-transformers redditcleaner psaw pandas loguru distinctipy colour tqdm praw asyncpraw rake-spacy hdbscan
!python -m spacy download en_core_web_trf
!python -m spacy download en_core_web_sm

Mounted at gdrive
2021-08-09 21:45:04.874143: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 460.2 MB 8.5 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
2021-08-09 21:45:42.383677: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.6 MB 168 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import os
path = "/content" # /content is pretty much the root. you can choose other path in your colab workspace
os.chdir(path)

In [3]:
# 1. Download the repo and set it as the current directory
!pip uninstall bertopic --yes
!rm -fr BERTopic
!git clone https://github.com/MasslessAI/BERTopic.git
%cd BERTopic

# 2. install the project/module
!python setup.py install

# 3. Add the project directory to the path
import os, sys
sys.path.append(os.getcwd())

%cd ..

Found existing installation: bertopic 0.8.1
Uninstalling bertopic-0.8.1:
  Successfully uninstalled bertopic-0.8.1
Cloning into 'BERTopic'...
remote: Enumerating objects: 1125, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1125 (delta 111), reused 77 (delta 71), pack-reused 951
Receiving objects: 100% (1125/1125), 7.02 MiB | 14.54 MiB/s, done.
Resolving deltas: 100% (604/604), done.
/content/BERTopic
running install
running bdist_egg
running egg_info
creating bertopic.egg-info
writing bertopic.egg-info/PKG-INFO
writing dependency_links to bertopic.egg-info/dependency_links.txt
writing requirements to bertopic.egg-info/requires.txt
writing top-level names to bertopic.egg-info/top_level.txt
writing manifest file 'bertopic.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'bertopic.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_p

# Scrap Reddit Data

Using pushshift api to quickly search for submissions where title contains **question-indicative phrases/words**. Only retrieve submissions whose **num_comments > 1**, this filters out most of the ads.

refer to [Content Ideas From Reddit](https://timothywangdev.github.io/knowledge/Content%20Marketing/content-ideas-from-reddit)

In [4]:
import datetime as dt
from datetime import date, datetime
from loguru import logger
import pandas as pd
from psaw import PushshiftAPI
import time
import os
import redditcleaner
import re
import base64
import IPython
from distinctipy import distinctipy
import random
from colour import Color
import praw
import pprint
# import asyncpraw


r = praw.Reddit(
  client_id="_tkKH5nva25sWQ",
  client_secret="cGMI8KyRQlopBTx7vGAEbgCDF6RzwQ",
  user_agent="analysis",
  check_for_async=False
)


api = PushshiftAPI(r)

QUESTION_WORDS = [
  "what",
  "when",
  "where",
  "who",
  "whom",
  "which",
  "whose",
  "why",
  "how",
  "wonder",
  "want",
  "is anyone",
  "does anyone",
  "any tips",
  "advice",
  "suggestion",
  "suggestions",
  "suggest",
  "ideas on",
  "need help",
  "needs help",
  "need your help",
  "serious help",
  "please help",
  "challenge",
  "challenges",
  "can't stand",
  "struggle",
  "struggling",
  "can't figure out",
  "help me",
  "hardest part",
  "would appreciate",
  "would really appreciate",
  "any guidance",
  "no idea",
  "confused with",
  "new to",
  "is there any way"
]

# (Optional) Do some experiments on question words and psaw filters

In [ ]:

# do some exps on question words and psaw filters
_QUESTION_WORDS=["ideas on", "struggle with", "suggestion"]
title_query = '|'.join(map(lambda x: '"{}"'.format(x), _QUESTION_WORDS))
print(title_query)

gen = list(
        api.search_submissions(
            title=title_query,
            is_self=True,
            is_original_content=True, 
            subreddit='content_marketing',
            num_comments=">1",
            filter=['title', 'selftext', 'author', 'permalink', 'num_comments', 'score', 'total_awards_received',
                    'upvote_ratio'],
            sort='asc', sort_type='created_utc', limit=500))

titles = [itemfor item in gen]
print(titles)

# generate distinct hsl colors
bg_color_list = distinctipy.get_colors(50)
inverted_color_list = distinctipy.invert_colors(bg_color_list)
text_color_list = [distinctipy.get_text_color(color) for color in bg_color_list]

# convert colors to hex
bg_color_list = [Color(rgb=color).hex for color in bg_color_list]
inverted_color_list = [Color(rgb=color).hex for color in inverted_color_list]
text_color_list = [Color(rgb=color).hex for color in text_color_list]

def colored(word, color_idx=0):
  color = bg_color_list[color_idx]
  return '<span style="display: inline-block; font-weight: 600; font-size: 16px; border-radius: 15%; padding-left: 5px; padding-right: 5px; margin: 4px 2px; background-color: {}; color:{};">{}</span>'.format(bg_color_list[color_idx], text_color_list[color_idx], word)
  
colored_titles = []
for title in titles:
  _title = title.lower()
  for idx, question_word in enumerate(_QUESTION_WORDS):
    _title = _title.replace(question_word, colored(question_word, color_idx = idx))
  colored_titles.append('<li>' + _title + "</li>")

IPython.display.HTML("<ol>" +' '.join(colored_titles) + "</ol>")

"ideas on"|"struggle with"|"suggestion"
submission(author='aaronmarsden', created_utc=1516890417, num_comments=8, permalink='/r/content_marketing/comments/7swe37/what_do_you_struggle_with_most_when_writing_and/', score=3, selftext='I\'ve been writing online for over 2 years. \n\nBut when I first started, **I had no fucking idea what I was doing.** Everything was foreign to me.\n\n*"How do I write enjoyable blog posts? How do I find my voice? How do I brainstorm topics? How do I conduct research?"*\n\nI hated not having answers, feeling lost. And that\'s why I want to help you. \n\nSo, what do you struggle with? \n\nI\'ll do my best to answer anything you have for me. Consider this my introduction to the Content Marketing subreddit. :)', title='What do you struggle with most when writing and creating content for your business?', created=1516890417.0, d_={'author': 'aaronmarsden', 'created_utc': 1516890417, 'num_comments': 8, 'permalink': '/r/content_marketing/comments/7swe37/what_do_you

Only retrieve submissions that are text submissions, and filter out deleted submissions and submissions whose author is banned.

We keep the following fields:

* title
* selftext
* author
* permalink
* num_comments <font color='blue'>*</font>
* score <font color='blue'>*</font>
* upvote_ratio <font color='blue'>*</font>
* total_awards_received <font color='blue'>*</font>

<font color='blue'>*</font> *Used for future ranking purpose*

<mark>TODO</mark> 
 - [ ] Use praw to get latest metadata (pushshift only updates data in for the first 24hr)


In [ ]:
start_epoch = int(dt.datetime(2018, 1, 1).timestamp())
#end_epoch = int(dt.datetime(2020, 3, 1).timestamp())
end_epoch = int(time.time())
total = 0

SUBREDDIT = 'startups'
DATA_FILE_NAME = gdrive_path + '/reddit_submission_{}_{}_{}.tsv'.format(
    SUBREDDIT, datetime.fromtimestamp(start_epoch).strftime("%Y_%m_%d"),
    datetime.fromtimestamp(end_epoch).strftime("%Y_%m_%d"))

title_query = '|'.join(map(lambda x: '"{}"'.format(x), QUESTION_WORDS))
while True:
    gen = list(
        api.search_submissions(
            after=start_epoch, 
            before=end_epoch,
            title=title_query,
            is_self=True,
            is_original_content=True,
            subreddit=SUBREDDIT,
            # note the num_comments is only updated for the first 24hr
            # may need to use praw to get latest meta data
            num_comments=">1",
            filter=['title', 'selftext', 'author', 'permalink', 'num_comments', 'score', 'total_awards_received',
                    'upvote_ratio'],
            sort='asc', 
            sort_type='created_utc', 
            limit=500))

    if len(gen) == 0:
        break

    def submission_filter(submission):
        if 'title' not in submission:
            return False
        if 'selftext' not in submission:
            return False
        if len(submission['selftext']) == 0:
            # if submission is deleted, the psaw returns empty str
            return False
        if 'author' not in submission:
            return False
        if submission['author'] == "[deleted]":
            return False
        if any(submission['selftext'] == x for x in ["[removed]", "[deleted]"]):
            return False
        return True

    def prepare_data(data):
        # some of the fields may be missing
        # must manually set an init value to avoid
        # generating invalid csv
        _data = {
            'title': '',
            'selftext': '',
            'author': '',
            'permalink': '',
            'num_comments': 0,
            'score': 0,
            'total_awards_received': 0,
            'upvote_ratio': 1.0,
            'created_utc': None
        }

        for key in _data:
            if key in data and data[key] is not None:
                _data[key] = data[key]

        return _data

    
    items = []
    keys = ['title', 'selftext', 'author', 'permalink', 'num_comments', 'score', 'total_awards_received', 'upvote_ratio', 'created_utc']
    for _item in gen:
      items.append({
        'title': _item.title,
        'selftext': _item.selftext,
        'author': _item.author,
        'permalink': _item.permalink,
        'num_comments': _item.num_comments,
        'score': _item.score,
        #'total_awards_received': _item.total_awards_received,
        'upvote_ratio': _item.upvote_ratio,
        'created_utc': _item.created_utc
      })

    items = map(prepare_data, items)
    # items = map(prepare_data, [item.d_ for item in gen])

    items = list(filter(submission_filter, items))
    df = pd.DataFrame(items)

    # clean data
    def clean(text):
        text = text.lower()
        # remove reddit styles
        text = redditcleaner.clean(
            text, quote=False, bullet_point=False, link=False, strikethrough=False, spoiler=False, code=False,
            superscript=False, table=False)

        # refer to https://towardsdatascience.com/cleaning-text-data-with-python-b69b47b97b76
        # Remove unicode characters
        text = text.encode('ascii', 'ignore').decode()

        # Remove Hashtags
        text = re.sub("#\S+", " ", text)

        # Remove markdown links
        text = re.sub(r"\[(.+)\]\(.+\)", r"\1", text)

        # Remove other urls
        text = re.sub(r"http\S+", " ", text)

        # remove text inside brackets
        text = re.sub("\(.*?\)"," ", text)
        text = re.sub("\[.*?\]"," ", text)

        # remove quotes
        # remove brackets
        # remove semicolon
        text = re.sub(r'[\t()[\]\"*:\\]',' ', text)

        # remove non-ascii chars
        text = re.sub(r"[^\x00-\x7F]+",' ', text)

         # Replace the over spaces# if submission is deleted, the psaw returns NaN, must check if 
            # it's a valid string
        text = re.sub('\s{2,}', " ", text)

        return text

    if len(df) > 0:
      df['title'] = df['title'].map(clean)
      df['selftext'] = df['selftext'].map(clean)

      if not os.path.isfile(DATA_FILE_NAME):
          df.to_csv(DATA_FILE_NAME, sep='\t', header='column_names', index=False, quoting=3)
      else:  # else it exists so append without writing the header
          df.to_csv(DATA_FILE_NAME, sep='\t', mode='a', header=False, index=False, quoting=3)

      start_epoch = int(items[-1]['created_utc'])
      total += len(items)

      logger.info('Added {} Total {} Last created_utc {}'.format(
        len(items), total, date.fromtimestamp(start_epoch)))

      time.sleep(3)
    else:
      break

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 502
  warnings.warn("Got non 200 code %s" % response.status_code)
/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")
2021-08-04 21:24:30.930 | INFO     | __main__:<module>:145 - Added 272 Total 272 Last created_utc 2018-04-10
2021-08-04 21:25:12.735 | INFO     | __main__:<module>:145 - Added 265 Total 537 Last created_utc 2018-07-15
2021-08-04 21:25:34.089 | INFO     | __main__:<module>:145 - Added 220 Total 757 Last created_utc 2018-10-16
2021-08-04 21:25:56.174 | INFO     | __main__:<module>:145 - Added 148 Total 905 Last created_utc 2018-12-13
2021-08-04 21:26:28.683 | INFO     | __main__:<module>:145 - Added 140 Total 1045 Last created_utc 2019-02-02
2021-08-04 21:26:43.861 | INFO     | __main__:<module>:145 - Added 158 Total 1

# Load Scrapped Reddit Data

Load submissions, filter out ads, and classified them by wh-words

From observation, some **submissions are ads**, which often contain the following symbols/phrases

```
'-', ':', ';' 
your business(es)
help you
case study
ALL CAPITAL chars
how i
dollar symbol $ (e.g turn $100 into 100k)
step-by-step
here is how
here's how
part 1/2/3
top [number] things
[number] reasons
ultimate guide
cheat sheet
cheatsheet
infographic
by [year]
ama
```

<mark>TODO</mark> 
 - [x] Add more ad-indicative phrases
 - [x] Use num_comments to filter ads

In [5]:
df = pd.read_csv(gdrive_path +"/reddit_submission_startups_2018_01_01_2021_08_04.tsv", sep='\t', quoting=3)

# data cleanup
print('before clean-up # rows: {}'.format(len(df)))
cleaned_rows = []

AD_INDICATIVE_PHRASES = [
  "your business", 
  "your businesses", 
  "help you", 
  "case study"
  "how i",
  "$",
  "step-by-step",
  "here is how",
  "here's how",
  "part 1",
  "part 2",
  "part 3",
  "ultimate guide",
  "cheatsheet",
  "infographic",
  "ama"
]

for index, row in df.iterrows():
    row_title = row['title'].lower()
    if not any(x in ['-', ':', ';'] for x in row_title) and not any(phrase in row_title for phrase in AD_INDICATIVE_PHRASES) and "?" in row_title:
        '''
        1. must contain '?'
        2. can only contain alphanumeric, punctuations and space
        3. should not contain '-', ':', ';' which indicates ads
        '''
        cleaned_rows.append(index)

df = df[df.index.isin(cleaned_rows)].reset_index()
print('after clean-up # rows: {}'.format(len(df)))

print(df.head(5).to_markdown())

before clean-up # rows: 5013
after clean-up # rows: 2889
|    |   index | title                                                                      | selftext                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

# (Optional) Tokenization Tunning

In [ ]:
import spacy

spacy.require_gpu()
nlp = spacy.load("en_core_web_trf")

In [ ]:
#nlp.add_pipe("merge_noun_chunks")
# nlp.add_pipe("merge_entities")

<function spacy.pipeline.functions.merge_noun_chunks>

In [ ]:
from spacy.matcher import Matcher
from spacy import displacy
from rake_spacy import Rake

idx = 5
text = df['title'][idx] + ' ' + df['selftext'][idx]

text = "what would the general lee of the north be?  "
doc = nlp(text)

for token in doc:
  print(token, token.pos_)

# merge entities and noun phrases (without DET)
def valid_start_pos(doc, span):
  start = span.start
  while start < span.end:
    if doc[start].pos_ not in ['DET', 'PRON']:
      break
    start += 1
  return start

# remove prefix tokens that are either DET or PRON
# remove spans with less than 2 tokens
# remove spans with 0 tokens after removing prefix
noun_phrase_spans = list(filter(lambda x: x.start < x.end, [doc[valid_start_pos(doc, np_span):np_span.end] for np_span in doc.noun_chunks if np_span.end-np_span.start > 1]))
entities_spans = list(filter(lambda x: x.start < x.end, [doc[valid_start_pos(doc, ent_span):ent_span.end] for ent_span in doc.ents if ent_span.end-ent_span.start > 1]))

spans = spacy.util.filter_spans(noun_phrase_spans+entities_spans)
with doc.retokenize() as retokenizer:
  for span in spans:
    retokenizer.merge(span)

print('\n\n')
for token in doc:
  print(token, token.pos_, token.is_stop)

print('\n\n')
print(' '.join(['_'.join(token.lemma_.split())for token in doc if not token.is_stop]))


# displacy_image = displacy.render(nlp(text2), jupyter = True, style = 'ent')


what PRON
would AUX
the DET
general ADJ
lee NOUN
of ADP
the DET
north NOUN
be VERB
? PUNCT
  SPACE



what PRON True
would AUX True
the DET True
general lee NOUN False
of ADP True
the DET True
north NOUN False
be VERB True
? PUNCT False
  SPACE False



general_lee north ? 


# Classified submissions by wh-words in title, extract keywords and aggregate stats

In [ ]:
!pip install sacremoses

In [6]:
import pprint
import spacy
from tqdm import tqdm 
from rake_spacy import Rake

spacy.require_gpu()
nlp = spacy.load("en_core_web_trf")
#kw_model = KeyBERT(model='paraphrase-MiniLM-L6-v2')
rake = Rake(nlp=nlp)

title_cat = []
keywords = []
lemmatized_combined_text = []

df['combined_text'] = df['title'] +' '+ df['selftext']
#docs = list(nlp.pipe(df['combined_text']))
for index, row in tqdm(df.iterrows(), position=0, leave=True, total=len(df)):
    title_cat.append('NO_WH_WORD')  
    for wh_word in QUESTION_WORDS:
        if wh_word in row['title'].lower():
            title_cat[-1] = wh_word
            break

df['title_cat'] = title_cat
#df['keywords'] = keywords

cat_stats = {
    'cat': QUESTION_WORDS,
    'num_docs': [],
    'total_score': [],
    'total_comments': []
}

# print out number of docs per category
for wh_word in QUESTION_WORDS:
    _df = df[df['title_cat'] == wh_word]
    cat_stats['num_docs'].append(len(_df))
    cat_stats['total_score'].append(_df['score'].sum())
    cat_stats['total_comments'].append(_df['num_comments'].sum())
  
cat_stats_df = pd.DataFrame(data=cat_stats)

print('\n', cat_stats_df.to_markdown())
  

100%|██████████| 2889/2889 [00:00<00:00, 9580.00it/s]


 |    | cat                     |   num_docs |   total_score |   total_comments |
|---:|:------------------------|-----------:|--------------:|-----------------:|
|  0 | what                    |        846 |         18855 |            19007 |
|  1 | when                    |        155 |          2706 |             2514 |
|  2 | where                   |        162 |          2672 |             3244 |
|  3 | who                     |         66 |          1496 |             1556 |
|  4 | whom                    |          0 |             0 |                0 |
|  5 | which                   |         66 |           687 |              982 |
|  6 | whose                   |          0 |             0 |                0 |
|  7 | why                     |         66 |          2166 |             2534 |
|  8 | how                     |       1351 |         24993 |            25411 |
|  9 | wonder                  |          1 |            10 |                4 |
| 10 | want               

# Pick a wh-word category

In [10]:
# df_selection = df[df['title_cat'] == "what"].copy().reset_index()
df_selection = df.copy().reset_index()
print(df_selection.head(10).to_markdown())

|    |   level_0 |   index | title                                                                                                        | selftext                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

# Keywords Analysis

In [8]:
from gensim.corpora import Dictionary
from collections import defaultdict

# Create a id2word dictionary
_dict = Dictionary(df_selection['keywords'])
print('dict size (unfiltered) {}'.format(len(_dict)))


# Filtering Extremes
_dict.filter_extremes(no_below=2, no_above=.99)
_dict.compactify()
print('dict size {}'.format(len(_dict)))

keywords_stats = defaultdict(lambda: {
    'token': '',
    'num_comments': 0,
    'score': 0,
    'doc_freq': 0,
    'docs': []
})

token2doc = defaultdict(list)
for idx, row in df_selection.iterrows():
    for token in row['keywords']:
        # ignore tokens not in dict
        if token not in _dict.token2id: continue
        token2doc[token].append(idx)
        keywords_stats[token]['num_comments'] += row['num_comments']
        keywords_stats[token]['score'] += row['score']

for token, id in _dict.token2id.items():
    keywords_stats[token]['token']=token
    keywords_stats[token]['doc_freq'] = _dict.dfs[id]
    keywords_stats[token]['docs'] = token2doc[token]

keywords_stats_df = pd.DataFrame(list(keywords_stats.values()))

# sort by num_comments
keywords_stats_df =keywords_stats_df.sort_values(by=['num_comments'], ascending=False)

print(keywords_stats_df[['token', 'num_comments', 'score', 'doc_freq']].head(50).to_markdown())

KeyError: ignored

# Topic Modeling with BERTopic

In [11]:
!pip install --upgrade --user numpy
!pip install hdbscan

     |████████████████████████████████| 15.7 MB 128 kB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.21.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
bertopic 0.8.1 requires plotly<4.14.3,>=4.7.0, but you have plotly 4.4.1 which is incompatible.


In [8]:
import sys
sys.path.insert(0,'/content/BERTopic')

In [17]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from bertopic.backend import BaseEmbedder

sentence_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# we must use BaseEmbedder as base class, otherwise BERTopic will use default 
# SentenceTransformer("paraphrase-MiniLM-L6-v2")
class CustomSentenceModel(BaseEmbedder):
  def __init__(self, embedding_model):
    super().__init__()
    self.embedding_model = embedding_model
  
  def embed(self, documents, verbose):
    embeddings = self.encode(documents, show_progress_bar=verbose)
    return embeddings

  def encode(self,documents,show_progress_bar = False):
        # need to split phrases into individual words in order to use general
        # sentence embedding models
        documents = list(map(lambda doc: re.sub('_', ' ', doc), documents))
        embeddings = self.embedding_model.encode(documents, show_progress_bar=show_progress_bar)
        return embeddings

def preprocess_text(documents_by_cluster):
  print('using own process...')
  processed_documents_by_cluster = []
  for document in documents_by_cluster:
    docs = list(nlp.pipe(document.split('\t')))
    processed_doc = []
    for doc in docs:
       # merge entities and noun phrases (without DET)
      def valid_start_pos(doc, span):
        start = span.start
        while start < span.end:
          if doc[start].pos_ not in ['DET', 'PRON']:
            break
          start += 1
        return start

      # remove prefix tokens that are either DET or PRON
      # remove spans with less than 2 tokens
      # remove spans with 0 tokens after removing prefix
      noun_phrase_spans = list(filter(lambda x: x.start < x.end, [doc[valid_start_pos(doc, np_span):np_span.end] for np_span in doc.noun_chunks if np_span.end-np_span.start > 1]))
      entities_spans = list(filter(lambda x: x.start < x.end, [doc[valid_start_pos(doc, ent_span):ent_span.end] for ent_span in doc.ents if ent_span.end-ent_span.start > 1]))

      spans = spacy.util.filter_spans(noun_phrase_spans+entities_spans)
      with doc.retokenize() as retokenizer:
        for span in spans:
          retokenizer.merge(span)
      
      processed_text = ' '.join(['_'.join(token.lemma_.split(' ')) for token in doc if not token.is_stop])
      processed_doc.append(processed_text)
    processed_documents_by_cluster.append('\t'.join(processed_doc))
  return processed_documents_by_cluster

_docs = df_selection['combined_text']

vectorizer = CountVectorizer(ngram_range=(1,2), max_df=0.95)

topic_model = BERTopic(preprocess_text=preprocess_text, vectorizer_model=vectorizer, embedding_model=CustomSentenceModel(sentence_model), nr_topics="auto", calculate_probabilities=True, min_topic_size=5)

print(len(_docs))
topics, probabilities = topic_model.fit_transform(_docs)
#topics, probabilities = topic_model.fit_transform(df_selection['lemmatized_combined_text'])
topic_info_df = topic_model.get_topic_info()
topic_tokens = [[token[0] for token in topic_model.get_topic(topic=row['Topic'])] for idx, row in topic_info_df.iterrows()]
topic_info_df['Tokens']=topic_tokens
print(topic_info_df[['Topic', 'Count', 'Tokens']].to_markdown())

2889


2021-08-09 22:18:56,264 - BERTopic - Transformed documents to Embeddings
2021-08-09 22:19:12,831 - BERTopic - Reduced dimensionality with UMAP
2021-08-09 22:19:14,188 - BERTopic - Clustered UMAP embeddings with HDBSCAN


using own process...
using own process...


2021-08-09 22:21:21,944 - BERTopic - Reduced number of topics from 103 to 62


|    |   Topic |   Count | Tokens                                                                                                                                                                                                                                                                 |
|---:|--------:|--------:|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 |      -1 |    1240 | ['startup', 'company', 'start', 'idea', 'try', 'business', 'ask', 'founder', 'create', 'user']                                                                                                                                                                         |
|  1 |       0 |     243 | ['startup', 'start', 'business', 'ask', 'hire', 'founder', 'get', 'company', 'job', 'start busine

# Dump results to JSON (Optional)

In [24]:

# dump topic info
topic_info_df.to_json('report_topic_info.json')

# add topic data to doc then dump
doc_topic = []
doc_topic_prob = []
for i in range(len(topics)):
  doc_topic.append(topics[i])
  doc_topic_prob.append(probabilities[i][topics[i]])

df_selection['topic'] = doc_topic
df_selection['topic_prob'] = doc_topic_prob

print(df_selection.head(20).to_markdown())
df_selection.to_json('report_submissions.json')


|    |   level_0 |   index | title                                                                                                        | selftext                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [12]:
from collections import defaultdict

docs_by_topics = defaultdict(list)

for index, topic in enumerate(topics):
  docs_by_topics[topic].append({
      'docIdx': index,
      'prob': probabilities[index][topic],
      'num_comments':df_selection['num_comments'][index],
      'score': df_selection['score'][index],
      'title': df_selection['title'][index],
      #'keywords': df_selection['keywords'][index]
  })

# sort doc by their topic prob
for index, topic in enumerate(topics):
  docs_by_topics[topic].sort(key=lambda x: x['score'], reverse=True)


In [ ]:

for idx, row in topic_info_df.iterrows():
  print(row.to_frame().T)
  topic_docs_df = pd.DataFrame(docs_by_topics[row['Topic']])
  print(topic_docs_df.head(10).to_markdown())



In [14]:
topics_over_time = topic_model.topics_over_time(df_selection['title'], topics, df_selection['created_utc'], nr_bins=20)
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

using own process...
using own process...
using own process...
using own process...
using own process...
using own process...
using own process...
using own process...
using own process...
using own process...
using own process...
using own process...
using own process...
using own process...
using own process...
using own process...
using own process...
using own process...
using own process...
using own process...


ValueError: ignored

In [15]:
topic_model.visualize_topics()

RuntimeError: ignored

In [16]:
topic_model.visualize_heatmap()

TypeError: ignored

In [ ]:
topic_model.visualize_hierarchy()